In [60]:
import pandas as pd
import os
import numpy as np

In [145]:
df = pd.read_parquet("../data_files/metadata_industry.parquet")

In [152]:
dfx = pd.read_parquet("../data_files/data_industry.parquet")

In [158]:
dfx['unixReviewTime'].astype(int).max()

1535328000

In [146]:
df.columns

Index(['also_buy', 'also_view', 'asin', 'brand', 'category', 'date',
       'description', 'details', 'feature', 'fit', 'image', 'main_cat',
       'price', 'rank', 'similar_item', 'tech1', 'tech2', 'title'],
      dtype='object')

In [148]:
# Only keep necessary columns
df = df[['asin', 'title', 'brand', 'category', 'description', 'details', 'feature']]

In [102]:
# Get a pandas series of all the categories and print the 200 most frequent
categories = pd.Series(df['category'].apply(pd.Series).stack().reset_index(drop=True))
pd.set_option('display.max_columns', None)
print(categories.value_counts().head(300))

Software                                                                                                                                                                       22497
Education & Reference                                                                                                                                                           3099
Digital Software                                                                                                                                                                2552
Business & Office                                                                                                                                                               2429
Children's                                                                                                                                                                      2295
                                                                                               

In [ ]:
# From eye inspection, we see the following related categories for Antivirus
# Antivirus & Security
# Antivirus
# Internet Security Suites
# Antivirus &amp; Security
# Internet Security

# For this reason, we are going to use two keywords for extracting our brands of interest: 
# "antivirus" and "security"

In [149]:
# Convert all data to string, even the rows in 'category' that have data type numpy.ndarray
df2 = df.applymap(str).copy().reset_index(drop=True)

In [120]:
index_antivirus = df2['category'].str.lower().str.contains('antivirus|security')

In [121]:
print(df2.loc[index_antivirus, 'brand'].value_counts().head(200))

brand
Symantec                                             238
McAfee                                               127
Trend Micro                                           94
Webroot                                               70
Bitdefender                                           67
Kaspersky                                             61
ESET                                                  46
Panda Software                                        41
AVG Technologies                                      39
ThreatTrack Security                                  31
Intego                                                30
Computer Associates                                   27
AVG USA Technologies Inc.                             24
AVG                                                   19
Avanquest                                             18
Encore                                                17
Avira                                                 12
Zone Labs                

In [150]:
df2.columns

Index(['asin', 'title', 'brand', 'category', 'description', 'details',
       'feature'],
      dtype='object')

In [151]:
example =df2.loc[(~df2['category'].str.lower().str.contains('virus|security|firewall|protection'))
        & (~df2['title'].str.lower().str.contains('virus|security|firewall|protection'))
        & (~df2['description'].str.lower().str.contains('virus|security|firewall|protection'))
        & (~df2['details'].str.lower().str.contains('virus|security|firewall|protection'))
        & (~df2['feature'].str.lower().str.contains('virus|security|firewall|protection'))
        & (df2['brand'].str.lower().str.contains('avg|symantec|mcafee')), :]
example

,asin,title,brand,category,description,details,feature
616,B006GR1GWM,PC TuneUp 2012 - for up to 3 Users,AVG Technologies,['Video Games' 'PC' 'Games'],"[""Restore your PC to peakperformanceProductInf...","{'\n Item Weight: \n ': None, '\n Pac...",[]
896,B001HU12UO,"pcAnywhere Host and Remote 12.5, 1 User"" />",Symantec,['Software' 'Business & Office' 'Communication...,['Symantec is announcing Symantec pcAnywhere 1...,"{'\n Item Weight: \n ': None, '\n Pac...",['1 USER CD RET']
897,B001HTZBP2,Pcanywhere Host and Remote 12.5 1 User CD Ret...,Symantec,['Software' 'Business & Office' 'Communication...,['SYMANTEC SYMC PCANY HOST&REMOTE 12.5UR CD RE...,"{'\n Item Weight: \n ': None, '\n Pac...",['1 USER CD RET MM']
1227,B001UL5K0S,Symantec Ghost Solution Suite V. 2.5,Symantec,['Software' 'Utilities' 'Cross Platform'],"['<div class=""aplus""><div class=""two-third-col...","{'\n Item Weight: \n ': None, '\n Pac...","[""Enhance and leverage Microsoft's own best pr..."
1228,B001UL5K2G,Symantec Pcanywhere V.12.5 Host - Remote Manag...,Symantec,['Software' 'Business & Office' 'Communication...,"[""The world's leading remote control solution....","{'\n Item Weight: \n ': None, '\n Pac...",['Connect to remote devices simply and securel...
1689,B002RS8LRM,Norton Utilities Premier 14.5 1-User/3PC [Old ...,Symantec,['Software' 'Utilities' 'PC Maintenance'],['NORTON UTILITIES PREMIER EDITION 14.5 EN 1 U...,"{'\n Item Weight: \n ': None, '\n Pac...",[]
1771,B002X8V38A,Norton Online Backup 2.0 1 User / 5 GB,Symantec,['Software' 'Utilities' 'Backup'],['Norton Online Backup automatically backs up ...,"{'\n Item Weight: \n ': None, '\n Pac...",['Keeps all of your irreplaceable digital phot...
1854,B0035O1J1M,Backup Exec 2010 Server,Symantec,['Software' 'Utilities' 'Backup'],['SYMANTEC BE2010 SRVR WIN ML DVD SRVR BNDL BP...,"{'\n Item Weight: \n ': None, '\n Pac...",[]
2022,B003M090VA,Act! 2.0 for Macintosh,Symantec,['Software' 'Business & Office' 'Office Suites'],['ACT is a database program designed especiall...,"{'\n Item Weight: \n ': None, '\n Pac...",[]
2699,B004VQR66K,System Recovery 2011 Desktop Edition,Symantec,['Software' 'Utilities' 'Backup'],['SYMANTEC SSR DSKTP2011WIN BNDL BUS PK BAS 12...,"{'\n Item Weight: \n ': None, '\n Pac...",['2006 edition enables you to backup desktops ...


In [142]:
example.loc[6199, 'title']

'McAfee LiveSafe - Total Protection Solution (Unlimited* Device License)'

In [65]:
s.apply(pd.Series).stack().reset_index(drop=True)

0    a
1    b
2    c
3    b
4    c
5    d
dtype: object